In [1]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2024-11-25 09:03:32--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  22.3MB/s    in 8.3s    

2024-11-25 09:03:42 (19.3 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1/255.,
)

test_datagen = ImageDataGenerator(
    rescale=1/255.
)

In [5]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size = 32,
    class_mode="categorical"
)

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [6]:
mobilenet_url = "https://kaggle.com/models/google/mobilenet-v2/TensorFlow2/100-224-feature-vector/1"

In [7]:
import tensorflow_hub as hub
import tf_keras as keras

In [8]:
feature_extractor_layer = hub.KerasLayer(
    mobilenet_url,
    trainable=False,
    input_shape=(224, 224, 3),
    name="feature_extraction_layer"
)

In [9]:
model = keras.Sequential([
    feature_extractor_layer,
    keras.layers.Dense(10, activation="softmax", name="output_layer")
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 1280)              2257984   
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 2270794 (8.66 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [11]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="Adam",
    metrics=["accuracy"]
)

In [12]:
model.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data)
)

Epoch 1/5
24/24 [==============================] - 18s 519ms/step - loss: 1.9262 - accuracy: 0.3427 - val_loss: 1.2579 - val_accuracy: 0.6192
Epoch 2/5
24/24 [==============================] - 10s 423ms/step - loss: 1.0225 - accuracy: 0.6893 - val_loss: 0.8790 - val_accuracy: 0.7376
Epoch 3/5
24/24 [==============================] - 12s 518ms/step - loss: 0.7338 - accuracy: 0.8067 - val_loss: 0.7341 - val_accuracy: 0.7740
Epoch 4/5
24/24 [==============================] - 11s 462ms/step - loss: 0.5830 - accuracy: 0.8533 - val_loss: 0.6652 - val_accuracy: 0.7888
Epoch 5/5
24/24 [==============================] - 11s 487ms/step - loss: 0.4894 - accuracy: 0.8800 - val_loss: 0.6475 - val_accuracy: 0.7972
